In [1]:
authorList = [4562, 551, 308, 123, 78] 
doc_id = 1210
loc = 1

In [2]:
parameters = {
    'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'samples': [50, 100, 150, 200, 250, 300],
    'dimensions': [50, 100, 200], # 300],
    'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
}
results = {
    'dropout': [0, 0, 0, 0, 0, 0],
    'samples': [0, 0, 0, 0, 0, 0],
    'dimensions': [0, 0, 0, 0],
    'iterations': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}
level = 'char'
dimensions = 100
for idxp, paralist in parameters.iteritems():
    if idxp == 'dropout':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = val
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[loc])))
            results[idxp].insert(idxl, predY[loc])
        
    
    if idxp == 'samples':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = val, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[loc])))
            results[idxp].insert(idxl, predY[loc])
        
    
    if idxp == 'dimensions':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = val,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[loc])))
            results[idxp].insert(idxl, predY[loc])
        
    
    if idxp == 'iterations':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = val,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[loc])))
            results[idxp].insert(idxl, predY[loc])
    
    from IPython.display import clear_output
    
    clear_output()

In [3]:
print results

{'iterations': [0.087741122, 0.0060175289, 0.0029063446, 0.0076062526, 0.00084979844, 0.0022158797, 0.00054135348, 0.00030015566, 0.062979318, 0.0094336066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'dropout': [0.068479441, 0.0027613589, 0.017534975, 0.0043732366, 0.016508469, 0.033291448, 0, 0, 0, 0, 0, 0], 'dimensions': [0.019572333, 0.0035010355, 0.019300185, 0, 0, 0, 0], 'samples': [0.21982607, 0.21109262, 0.061043505, 0.017348628, 0.013018345, 0.0068867509, 0, 0, 0, 0, 0, 0]}


In [4]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [5]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()